In [1]:
import os
import sentencepiece as spm

input_file = "data/petitions_splited_mecab.txt"
vocab_size = 23000

sp_model_root  = "data/sentencepiece"

if not os.path.isdir(sp_model_root):
    os.mkdir(sp_model_root)

sp_model_name        = 'tokenizer_%d'
sp_model_path        = os.path.join(sp_model_root, sp_model_name)
model_type           = 'unigram'
character_coverage   = 1.0
user_defined_symbols = '[BOS],[EOS]'

input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --user_defined_symbols=%s --model_type=%s --character_coverage=%s'
cmd = input_argument%(input_file, sp_model_path, vocab_size, user_defined_symbols, model_type, character_coverage)

spm.SentencePieceTrainer.Train(cmd)

In [2]:
sp = spm.SentencePieceProcessor()
sp.Load('{}.model'.format(sp_model_path))

tokens = sp.encode_as_pieces('나는 오늘 아침밥을 먹었다.')
ids = sp.encode_as_ids('나는 오늘 아침밥을 먹었다.')

print(ids)
print(tokens)

tokens = sp.decode_pieces(tokens)
ids = sp.decode_ids(ids)

print(ids)
print(tokens)

[67, 391, 792, 1541, 4357, 886, 236, 1697, 326, 929]
['▁나', '는', '▁오늘', '▁아침', '밥', '을', '▁먹', '었', '다', '.']
나는 오늘 아침밥을 먹었다.
나는 오늘 아침밥을 먹었다.


In [3]:
import itertools
from konlpy.tag import Mecab
from tqdm import tqdm

# mecab = Mecab().morphs
mecab = Mecab()

with open("data/petitions_2019-01", 'r') as f:
    data = f.readlines()
    
sublist   = [line.split('"content": "')[-1].split('", "num_agree": ')[0] for line in data]
sublist   = [doc.split(". ") for doc in sublist]
petitions = list(itertools.chain.from_iterable(sublist))
petitions[:10]


['국민과 소통하시고 자유롭고 행복한 나라를 만들기 위해 힘쓰고 계신 대통령께 존경과 찬사를 올립니다',
 '기해년 새해 복 많이 받으십시오',
 '저는 경북 울진군 북면 부구검성로 12번지에 살고 있는 북면발전협의회장 이희국이라고 합니다',
 '저는 8기의 원전이 가동․건설되고 있는 이곳 북면에 태어나 68년째 거주하고 있는 원전지역 주민입니다',
 '간절한 마음을 담아 대통령께 다음과 같이 호소 드립니다',
 '‘울진군민과 약속한 신한울 3,4호기 원전건설을 재개해 주십시오.’ 여태껏 단 한 번도 원전 건설을 원한 적 없는 제가 신한울 3,4호기 원전 건설을 청하는 까닭을 말씀드리겠습니다',
 '경상북도 동해안 최북단 울진군은 예부터 산과 바다, 계곡의 울창함이 보배처럼 아름답다하여 “울진(蔚珍)”이라는 지명을 간직하게 된 곳입니다',
 '이러한 곳에 1981년 원전사업의 시작으로 울진군에 북면(6기), 산포지구(6기), 직산지구(6기)가 원전 예정지역으로 지정되면서, 먼저 북면 부구리 지역에 원전 6기가 건설되었습니다',
 '해안선이 잘려나가고 마을 한복판에 고압 송전탑이 들어섰습니다',
 '어장이 파괴되고 지역 특산품에 방사능 꼬리표가 붙었습니다']

In [29]:
total_morphs = []

for sentence in tqdm(petitions):
    morph_sentence = []
    
    for word in sentence.split(" "):
        count = 0
        for token, pos in mecab.pos(word):
            if count > 0:
                if not pos.startswith("N"):
                    token = "##" + token
                    morph_sentence.append(token)
                else:
                    morph_sentence.append(token)
            else:
                morph_sentence.append(token)
                count += 1
        break
    total_morphs.append(morph_sentence)
    
total_morphs = list(itertools.chain.from_iterable(total_morphs))

100%|██████████| 155357/155357 [00:02<00:00, 66503.90it/s]


In [30]:
total_morphs[:100]

['국민',
 '##과',
 '기',
 '해년',
 '저',
 '##는',
 '저',
 '##는',
 '간절',
 '##한',
 '‘',
 '울진',
 '군민',
 '##과',
 '경상북도',
 '이러',
 '##한',
 '해안선',
 '##이',
 '어',
 '장이',
 '삶',
 '##의',
 '그러',
 '##던',
 '저',
 '##를',
 '울진',
 '군민',
 '##과',
 '그것',
 '##이',
 '울진군',
 '민은',
 '2010',
 '년',
 '하',
 '##지만',
 '울진',
 '군민',
 '##에게',
 '##는',
 '단',
 '그',
 '지난',
 '정부',
 '의',
 '경기',
 '##가',
 '건설',
 '기존',
 '종잣돈',
 '##을',
 '건설',
 '장비',
 '##를',
 '신',
 '한울',
 '존경',
 '##하',
 '##는',
 '저',
 '##는',
 '정치',
 '##적',
 '단지',
 '친애',
 '##하',
 '##는',
 '부디',
 '지역',
 '주',
 '민의',
 '지난',
 '대통령',
 '##은',
 '약속',
 '##을',
 '간곡히',
 '2019',
 '년',
 '북면',
 '발전',
 '협의',
 '회장',
 '2008',
 '년',
 '대한민국',
 '11',
 '년',
 '지난',
 '농업',
 '##경',
 '영체',
 '돌아온',
 '신고',
 '새',
 '아직',
 '대한민국',
 '존경']

In [31]:
with open("data/petition_proc_with_hash_code.txt", 'w') as f:
    f.writelines(total_morphs)

In [32]:
from tokenizers import BertWordPieceTokenizer, SentencePieceBPETokenizer, CharBPETokenizer, ByteLevelBPETokenizer

how_to_tokenize = BertWordPieceTokenizer

if str(how_to_tokenize) == str(BertWordPieceTokenizer):
    print('BertWordPieceTokenizer')
    tokenizer = BertWordPieceTokenizer(strip_accents=False, lowercase=False)
elif str(how_to_tokenize) == str(SentencePieceBPETokenizer):
    print('SentencePieceBPETokenizer')
    tokenizer = SentencePieceBPETokenizer()
elif str(how_to_tokenize) == str(CharBPETokenizer):
    print('CharBPETokenizer')
    tokenizer = CharBPETokenizer() 
elif str(how_to_tokenize) == str(ByteLevelBPETokenizer):
    print('ByteLevelBPETokenizer')
    tokenizer = ByteLevelBPETokenizer()
else:
    assert('select right tokenizer')

corpus_file    = ['data/petition_proc_with_hash_code.txt']  # data path
vocab_size     = 32000
limit_alphabet = 6000
output_path    = 'hugging_%d'%(vocab_size)
min_frequency  = 5

# Then train it!
tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,
               limit_alphabet=limit_alphabet,
               show_progress=True)

BertWordPieceTokenizer


In [33]:
sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

sentence = 'I want to go my hometown'
output = tokenizer.encode(sentence)
print(sentence)
print('=>idx   : %s'%output.ids)
print('=>tokens: %s'%output.tokens)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

나는 오늘 아침밥을 먹었다.
=>idx   : [447, 1946, 4318, 1045, 1958, 1928, 1957, 749, 2612, 1672, 17]
=>tokens: ['나', '##는', '오늘', '아', '##침', '##밥', '##을', '먹', '##었', '##다', '.']
=>offset: [(0, 1), (1, 2), (3, 5), (6, 7), (7, 8), (8, 9), (9, 10), (11, 12), (12, 13), (13, 14), (14, 15)]
=>decode: 나는 오늘 아침밥을 먹었다.

I want to go my hometown
=>idx   : [44, 90, 4079, 2045, 87, 1891, 4268, 80, 2306, 75, 1891, 2238, 5012, 7979, 2152]
=>tokens: ['I', 'w', '##an', '##t', 't', '##o', 'go', 'm', '##y', 'h', '##o', '##m', '##et', '##ow', '##n']
=>offset: [(0, 1), (2, 3), (3, 5), (5, 6), (7, 8), (8, 9), (10, 12), (13, 14), (14, 15), (16, 17), (17, 18), (18, 19), (19, 21), (21, 23), (23, 24)]
=>decode: I want to go my hometown



In [34]:
hf_model_path='data/tokenizer_model'

if not os.path.isdir(hf_model_path):
    os.mkdir(hf_model_path)

tokenizer.save_model(hf_model_path)

['data/tokenizer_model/vocab.txt']

In [35]:
from transformers import BertTokenizerFast

tokenizer_for_load = BertTokenizerFast.from_pretrained(hf_model_path,
                                                       strip_accents=False,
                                                       lowercase=False)

print('vocab size : %d' % tokenizer_for_load.vocab_size)
tokenized_input_for_pytorch = tokenizer_for_load("나는 오늘 아침밥을 먹었다.", return_tensors="pt")

print("Tokens (str)      : {}".format([tokenizer_for_load.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))

vocab size : 12432
Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아', '##침', '##밥', '##을', '먹', '##었', '##다', '.', '[SEP]']
Tokens (int)      : [2, 447, 1946, 4318, 1045, 1958, 1928, 1957, 749, 2612, 1672, 17, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [36]:
tokenizer_for_load.all_special_tokens # 추가하기 전 기본적인 special token

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']